In [ ]:
%config Completer.use_jedi=False

import sys
import glob
sys.path.append('../../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import rankdata

from refsimu_singletrait import reload_simulation
from refsimu_singletrait import microbe_pop_on


def get_line_type(fname):
    if (fname in ['microbes','monomers','substrates','degradation_enzymes']):
        res = '-'
    else:
        res = '--'
    return res

In [ ]:
microbe_pop_on()

In [ ]:
# Is is assumed here that a simulation has been run and saved in the directory save_dir. 

save_dir = './simutest_batch' # 
# save_dir = './simutest_interactive_CELLULOSEX1/'
eco = reload_simulation(save_dir, itime=-1)
# with itime=-1 ecosystem parameters are load from json file, but biomassfields are set to zero, which allows
# to avoid having biomassfields_dump hdf5 files present

In [ ]:
print(eco.microbes.dispersal_op.forced_probas)
eco.microbes.dispersal_op.get_dispersal_prob_distrib('bacteria')

In [ ]:
diagcollector = eco.get_diag_collector()

In [ ]:
tl_files = sorted(glob.glob('{}/timelines_dump_*.hdf5'.format(save_dir)))
print(tl_files)

In [ ]:
dump_file = tl_files[0]

In [ ]:
diagcollector.load_from_dumpfile(dump_file)

In [ ]:
tla_comp_space = diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input' not in t)
tla_comp_space_external =  diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input'  in t)
tla_recycling = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'stoechio_balance_recycler' in t)
tla_mortality = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'mortality_op' in t)
tla_bytype = diagcollector.get_timeline_arrays('sum_by_type')
tla_ncells = diagcollector.get_timeline_arrays('ncells')
tla_sub = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'substrates' in t)
tla_enz = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'degradation_enzymes' in t)
tla_mon = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'monomers' in t)
tla_mic = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'microbes' in t)
tla_mic_ntax = diagcollector.get_timeline_arrays('ntaxpercell')

In [ ]:
tla_clim = diagcollector.get_timeline_arrays('Climate')
fig, axes = plt.subplots(2,1, figsize=(12,4))
for k, ax in zip(tla_clim.keys(),axes):
    ax.plot(tla_clim[k]['times'],tla_clim[k]['values'])
    ax.set_xlabel('time')
    ax.set_ylabel(k)
    ax.grid()

In [ ]:
def get_line_type(fname):
    if (fname in ['microbes','monomers','substrates','degradation_enzymes']):
        res = '-'
    else:
        res = '--'
    return res

fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    tmp_sum = None
    for k, d in tla_comp_space.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum += d['values'][:, ia]
    for k, d in tla_comp_space_external.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum -= d['values'][:, ia]
    ax.plot(td,scal*tmp_sum,'.',color='black',lw=0.2,alpha=0.1,label='sum internal - sum sources')
    ax.grid()
    ax.legend(fontsize=8, ncol=3,loc=1, framealpha=0.5)
    ax.set_title('Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
d = tla_comp_space['Respiration_Growth_Induci_Maint']
ax.plot(d['times'],d['values'][:,0]*scal)
ax.grid()
ax.set_title('Respiration Growth Induci Maint')

In [ ]:
tla_enz_p = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'Degradation_Enzymes' in t)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_enz_p.items():
    ax.plot(d['times'],d['values'][:,0,0]*scal,label=k)
ax.grid()
ax.legend()

In [ ]:
tla_osmo_p = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'Osmolytes' in t)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_osmo_p.items():
    ax.plot(d['times'],d['values'][:,0,0]*scal,label=k)
ax.grid()
ax.legend()

In [ ]:
tla_upt_p = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'Uptake_Transporters' in t)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_upt_p.items():
    ax.plot(d['times'],d['values'][:,0,0]*scal,label=k)
ax.grid()
ax.legend()

In [ ]:
fig, ax = plt.subplots(1,1)
t, dat = tla_mic_ntax['microbes']['times'],tla_mic_ntax['microbes']['values']
ax.plot(t,dat[:,0], label='mean')
ax.fill_between(t,dat[:,0]-dat[:,1],dat[:,0]+dat[:,1], alpha=0.2)
ax.plot(t,dat[:,2], label='min')
ax.plot(t,dat[:,3], label='max')
ax.grid()
ax.set_xlabel('Time')
ax.set_ylabel('Live Taxa per grid cell')
ax.legend();
plt.show()
plt.close(fig)
#

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_sub.items():
        for isub, subname in enumerate(eco.substrates.names):
            if (isub < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,isub,ia],ls,label=subname)
    ax.legend()
    ax.grid()
    ax.set_title('Substrate Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(14,6))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_sub.items():
    dat = np.sum(d['values'], axis=-1)
    loctot = np.sum(dat, axis=-1)
    dat = dat / loctot[:,np.newaxis]
    for isub, subname in enumerate(eco.substrates.names):
        if (isub < 2):
            ls = '--'
        else:
            ls = '-'
        ax.plot(d['times'],dat[:,isub],ls,label=subname)
ax.legend(loc=1 ,ncol=6, )
ax.grid()
ax.set_title('Substrate Relative abundancy');


In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_enz.items():
        for ienz, enzname in enumerate(eco.degradation_enzymes.names):
            if (ienz < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,ienz,ia],ls,label=enzname)
    ax.legend(ncol=6, fontsize=6)
    ax.grid()
    ax.set_title('Enzymes Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_mon.items():
        for imon, monname in enumerate(eco.monomers.names):
            if (imon < 4):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,imon,ia],ls,label=monname)
    ax.legend()
    ax.grid()
    ax.set_title('Monomers Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_mon.items():
        for imon, monname in enumerate(eco.monomers.names):
            if (monname not in (['DeadMic','DeadEnz'])):
                continue
            if (imon < 4):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,imon,ia],ls,label=monname)
    ax.legend()
    ax.grid()
    ax.set_title('Monomers Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_recycling.items():
    fig, ax = plt.subplots(1,1, figsize=(12,6))
    for ia in range(3):
        ax.plot(d['times'],scal*d['values'][:,ia], label=['C','N','P'][ia])
    ax.legend()
    ax.set_title(k)
    ax.grid()

In [ ]:
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_mortality.items():
    fig, ax = plt.subplots(1,1, figsize=(12,6))
    for ia in range(3):
        ax.plot(d['times'],scal*d['values'][:,ia], label=['C','N','P'][ia])
    ax.legend()
    ax.set_title(k)
    ax.grid()

In [ ]:
scal = 1.0 / eco.microbes.grid_size
for k, d in tla_bytype.items():
    mtypes = eco.microbes.get_taxa_types()
    print(mtypes)
    fig, axes = plt.subplots(3,1, figsize=(14,12))
    for ia, ax in enumerate(axes):
        ax.set_title('{} {}'.format(k, ['C','N','P'][ia]))
        for itype, typ in enumerate(mtypes):
            ax.plot(d['times'],scal*d['values'][:,itype,ia],label=typ)
        ax.grid()
        ax.legend(fontsize=9)

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
# plt.imshow(tla_ncells['microbes'][1].T,aspect='auto', origin='lower',interpolation='None')
plt.pcolormesh(tla_ncells['microbes']['values'].T *scal, shading='flat', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon grid occupation fraction ')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
# plt.imshow(tla_ncells['microbes'][1].T,aspect='auto', origin='lower',interpolation='None')
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
sdat = np.sum(dat,axis=1)
dat = dat / sdat[:,np.newaxis]
plt.pcolormesh(dat.T, shading='flat', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Total mass Relative abundancy')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
# plt.imshow(tla_ncells['microbes'][1].T,aspect='auto', origin='lower',interpolation='None')
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
sdat = np.sum(dat,axis=1)
dat = dat / sdat[:,np.newaxis]
plt.pcolormesh(np.log10(dat.T), shading='flat', cmap='magma')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Total mass Relative abundancy (log)')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
ranks = rankdata(-dat,method='max', axis=1)
plt.imshow(-ranks.T, aspect='auto',origin='lower',interpolation='None', cmap='hot')
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon Total mass rank')
plt.colorbar();

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
# plt.imshow(tla_ncells['microbes'][1].T,aspect='auto', origin='lower',interpolation='None')
plt.pcolormesh((tla_ncells['microbes']['values'].T > 0).astype(float), shading='flat', cmap='gray', vmin=0,vmax=1)
plt.xlabel('time')
plt.ylabel('taxon id')
plt.title('Taxon grid survival ')
plt.colorbar();

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_mic.items():
        for itax, micname in enumerate(eco.microbes.names):
            if (itax%2 ==0):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,itax,ia],ls,label=micname)
    ax.legend(ncol=8, fontsize=6)
    ax.grid()
    ax.set_title('Taxa Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig = plt.figure(figsize=(14,8))
for itax in range(tla_ncells['microbes']['values'].shape[1]):
    if (np.max(tla_ncells['microbes']['values'][:,itax]) > 0):
        plt.plot(tla_ncells['microbes']['times'],tla_ncells['microbes']['values'][:,itax], label='{}'.format(itax))
plt.legend(fontsize=6, ncol=6)
plt.title('Taxa ncell > 50')
plt.xlabel('Time')
plt.ylabel('Number of occupied cells')
plt.grid()
